<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />

# Evaluación del proceso de entrenamiento<a id="top"></a>

<i><small>Autor: Alberto Díaz Álvarez<br>Última actualización: 2023-03-14</small></i></div>
                                                  

***

## Introducción

Es importante monitorizar el proceso de entrenamiento de un modelo para poder evaluar y mejorar su rendimiento. Durante el entrenamiento de una red neuronal es necesario ajustar numerosos hiperparámetros, como la tasa de aprendizaje, el tamaño del _batch_, el número de _epochs_, la topología de la red, etc.

Tensorboard es una herramienta útil para monitorizar el proceso de entrenamiento, ya que permite visualizar gráficamente diversas métricas que se recopilan durante el proceso de entrenamiento, como la evolución de la función de pérdida (esto es, el _loss_), la precisión de la red en los datos de entrenamiento y validación, el aprendizaje de las características de la red, entre otros. Además, Tensorboard también permite visualizar la estructura de la red neuronal y las distribuciones de los pesos y las activaciones de las capas, lo que puede ayudar a identificar posibles problemas en la arquitectura de la red.

## Objetivos

El objetivo general de este _notebook_ es el de enseñar cómo utilizar la herramienta **Tensorboard** para evaluar el entrenamiento de modelos de redes neuronales. Tras enseñar cómo arrancarlo dentro de nuestro notebook (es una herramienta externa que se puede usar independientemente desde la terminal), probaremos a detectar problemas de _exploding gradients_ y _vanishing gradients_.

## Bibliotecas y configuración

A continuación importaremos las bibliotecas que se utilizarán a lo largo del _notebook_.

In [2]:
import datetime

import tensorflow as tf

import os

os.environ['TENSORBOARD_BINARY'] = r'C:\Users\andro\AppData\Roaming\Python\Python39\Scripts\tensorboard.exe'#r'c:\users\andro\appdata\roaming\python\python39\site-packages\tensorboard'

***

## Nuestro modelo de ejemplo

Para este ejemplo, crearemos un modelo para resolver el problema MNIST que ya vimos en el ejercicio anterior.

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train / 255, x_test / 255
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## Cargando TensorBoard

Para inicializar tensorboard basta con cargarlo como módulo externo:

In [4]:
%reload_ext tensorboard

Luego, tenemos que crear un callback que irá actualizando los valores del modelo según vamos trabajando con él:

In [5]:
log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Una vez cargado el módulo y creado el _callback_ que usará `tensorboard`, pasamos a arrancarlo.

In [6]:
#%tensorboard --logdir $log_dir

%tensorboard --logdir $log_dir

En este momento, tensorboard está escuchando en el directorio de log (donde se están almacenando _loss_ y demás métricas) y actualizándose cada 30 segundos.

Ya podemos entrenar y ver cómo va evolucionando el entrenamiento de nuestro modelo en `tensorboard`:

In [31]:
history = model.fit(x_train, y_train, epochs=25, validation_split=0.1, verbose=0, callbacks=[tb_callback])

## Sobre los componentes

Ahora veremos los componentes principales que están disponibles en TensorBoard

### Grafos

Son elementos muy complicados de seguir según crece la complejidad del grafo.

![Vista del grafo completo de nuestro ejemplo](Images/graph.png "Vista del grafo completo de nuestro ejemplo")

La forma de conseguir que nos den una vista útil es hacer un trabajo previo de limpieza para estructurarlo.

Los grafos nos pueden ayudar enormemente tanto para comprender el modelo con el que estamos trabajando, como a detectar errores de la topología del modelo.

Algunas claves para interpretar el grafo:

1. Los nodos con el mismo color implica que pertenecen a la misma estructura. Los grises, sin embargo, indican que cada uno de los nodos es único.
2. Haciendo click en un nodo se ven más detalles
3. Existe un botón que nos permite ver dependencias de cualquier nodo: `trace_input`

### Resúmenes (_summaries_)

Son un tipo de operador especial de tensorflow. Al igual que existen operadores como las operaciones algebráicas (e.g. sumas, restas, ...), existen operadores que toman como entrada un tensor del grafo y ofrecen como salida un conjunto de datos "resumidos".

Por defecto, existen operadores resumen creados automáticamente (prácticamente todo gráfico que aparece aparte del grafo en TensorBoard es un operador de este tipo), aunque nosotros podemos crear tantos como necesitemos. Una vez estén creados, se volcarán en los logs, los cuales se leerán desde tensorboard.

Ahora veremos algunos de los operadores más comunes:

* `tf.summary.scalar`: Escriben valores individuales como la precisión, la pérdida, etcétera, mostrándolos en forma de gráfica.
* `tf.summary.image`: Muestra una imagen, lo cual es muy útil para identificar si las entradas son correctas o si un modelo generativo está produciendo imágenes como las esperadas.
* `tf.summary.audio`: Similar al operador anterior, pero para sonido.
* `tf.summary.histogram`: Útil para trazar el histograma de un tensor no escalar, el cual muestra cómo la distribución del valor del tensor cambia con el tiempo. En el caso de DNN se utiliza comúnmente para comprobar la distribución de los pesos y los sesgos, ayudando a detectar comportamientos irregulares en los parámetros de la red.

## Vanishing gradients

El gradiente es una medida de la dirección y la magnitud del cambio en la función de pérdida de la red neuronal, que se utiliza para ajustar los pesos de las conexiones de la red durante el proceso de entrenamiento.

Si el gradiente es muy pequeño, esto puede llevar a problemas de _vanishing gradients_, donde los pesos de las conexiones de la red se actualizan en pequeños saltos que pueden hacer que la red se estanque en un mínimo local y no pueda aprender patrones más complejos.

Vamos a hacer un ejercicio para identificar cómo nuestro modelo está sufriendo un problema de _vanishing gradients_. Terminaremos primero con el proceso anterior de `tensorboard`.

In [32]:
!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')

"kill" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


Ahora crearemos un modelo diferente y lanzaremos un nuevo tensorboard para evaluar el entrenamiento.

In [33]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(5, activation='tanh'),
    tf.keras.layers.Dense(5, activation='tanh'),
    tf.keras.layers.Dense(5, activation='tanh'),
    tf.keras.layers.Dense(5, activation='tanh'),
    tf.keras.layers.Dense(5, activation='tanh'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics = ['accuracy'])
model.summary()

log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

%tensorboard --logdir $log_dir

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 5)                 3925      
                                                                 
 dense_5 (Dense)             (None, 5)                 30        
                                                                 
 dense_6 (Dense)             (None, 5)                 30        
                                                                 
 dense_7 (Dense)             (None, 5)                 30        
                                                                 
 dense_8 (Dense)             (None, 5)                 30        
                                                                 
 dense_9 (Dense)             (None, 10)               

Vamos con el entrenamieto para ver cómo evoluciona.

In [34]:
history = model.fit(x_train, y_train, epochs=25, validation_split=0.1, verbose=0, callbacks=[tb_callback])

## Exploiding gradients

Si el gradiente es muy grande, esto puede llevar a problemas de _exploding gradients_, donde los pesos de las conexiones de la red se actualizan en grandes saltos que pueden hacer que la red no pueda converger a una solución óptima.

Comenzaremos finalizando el proceso de tensorboard anterior para poder lanzar posteriormente uno nuevo.

In [ ]:
!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')

Ya que este problema es poco común (aunque pasa) en redes poco profundas, en nuestro ejemplo actual es difícil conseguir el efecto deseado. Lo intentaremos trucando las entradas para que sean mayores de lo que deberían, forzando a que los valores que viajan por la red sean muy altos.

In [35]:
x_train, y_train = x_train * 10, y_train * 10

Ahora entrenaremos un modelo con estas entradas para que nos intente dar las salidas esperadas:

In [1]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'),
    tf.keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'),
    tf.keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'),
    tf.keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'),
    tf.keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics = ['accuracy'])
model.summary()

log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir $log_dir

NameError: name 'tf' is not defined

Ahora entrenaremos nuestro modelo.

In [37]:
history = model.fit(x_train, y_train, epochs=25, validation_split=0.1, verbose=0, callbacks=[tb_callback])

## Conclusiones

Este notebook ha sido intenso, pero en él hemos visto las principales diferencias de los dos tipos de problema que nos encontraremos en problemas de aprendizaje profundo: clasificación y regresión. Los modelos desarrollados para éstos son muy parecidos, varían básicamente en la salida y su cálculo del error.

También, para la evaluación de estos modelos hemos presentado algunas medidas, unas específicas para clasificación y otras para regresión. Hay algunas que no hemos explicado (e.g. entropía cruzada) pero hemos preferido quedarnos en las más comunes. Una cosa buena es que prácticamente todos los frameworks incluyen estas implementaciones, seguramente mucho mejor de lo que las podamos implementar nosotros. Sin embargo, es muy importante es saber cómo estamos midiendo y qué significan esas mediciones.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>